In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
csv_df = pd.DataFrame.from_csv("marijuana-street-price-clean.csv",header=0,index_col=False,sep=',')

In [4]:
#csv_df.head

In [5]:
csv_df_sort = csv_df.sort(columns=['State','date'])

In [6]:
# let us count the number of entries in each column

In [7]:
csv_df_sort.count()

State     22899
HighQ     22899
HighQN    22899
MedQ      22899
MedQN     22899
LowQ      12342
LowQN     22899
date      22899
dtype: int64

In [8]:
# we see that LowQ does not match the number of entries and this a common problem in time series analysis
# lets fill up the Nan value with the last known best value so that we can work on continuing time series analysis

In [9]:
csv_df_sort_fillna_ffil = csv_df_sort.fillna(method='ffill')

In [10]:
csv_df_sort_fillna_ffil.count()

State     22899
HighQ     22899
HighQN    22899
MedQ      22899
MedQN     22899
LowQ      22899
LowQN     22899
date      22899
dtype: int64

In [11]:
#now we have cleaned data set to work with

In [12]:
global_mean_HighQ = csv_df_sort_fillna_ffil['HighQ'].mean()

In [13]:
global_mean_HighQ

329.75985414210226

In [14]:
#now that we have the global mean, we can find how far the values of mean are for each state from global mean

In [15]:
print type(csv_df_sort_fillna_ffil.head())

<class 'pandas.core.frame.DataFrame'>


In [16]:
#group_by_state.get_group(('Alabama'))
# to check values for a given group
# now lets use agg function to get generate Mean,Median,Standard deviation,Variance and covariance for every state

In [17]:
basic_measures_state_wise_df = pd.DataFrame()

In [18]:
#lets get all the unique states in the data frame and construct the measures for each state
# normal method
'''
unique_states = pd.unique(csv_df_sort_fillna_ffil['State'].ravel())
for state in unique_states:
    state_df = csv_df_sort_fillna_ffil.loc[csv_df_sort_fillna_ffil['State'] == state]
    highq_df = state_df.groupby(['State'])['HighQ'].agg([np.mean,np.median,np.std,np.var,np.cov])
    highq_df.head()
    lowq_df = state_df.groupby(['State'],)['LowQ'].agg([np.mean,np.median,np.std,np.var,np.cov])
    highq_df = highq_df.rename(columns={'mean':'HighQ_Mean','median':'HighQ_Median','std':'HighQ_Std','var':'HighQ_Var','cov':'HighQ_Cov'})
    lowq_df = lowq_df.rename(columns={'mean':'LowQ_Mean','median':'LowQ_Median','std':'LowQ_Std','var':'LowQ_Var','cov':'LowQ_Cov'})
    highq_df['State'] = state
    lowq_df['State'] = state
    break
'''

"\nunique_states = pd.unique(csv_df_sort_fillna_ffil['State'].ravel())\nfor state in unique_states:\n    state_df = csv_df_sort_fillna_ffil.loc[csv_df_sort_fillna_ffil['State'] == state]\n    highq_df = state_df.groupby(['State'])['HighQ'].agg([np.mean,np.median,np.std,np.var,np.cov])\n    highq_df.head()\n    lowq_df = state_df.groupby(['State'],)['LowQ'].agg([np.mean,np.median,np.std,np.var,np.cov])\n    highq_df = highq_df.rename(columns={'mean':'HighQ_Mean','median':'HighQ_Median','std':'HighQ_Std','var':'HighQ_Var','cov':'HighQ_Cov'})\n    lowq_df = lowq_df.rename(columns={'mean':'LowQ_Mean','median':'LowQ_Median','std':'LowQ_Std','var':'LowQ_Var','cov':'LowQ_Cov'})\n    highq_df['State'] = state\n    lowq_df['State'] = state\n    break\n"

In [19]:
csv_df_sort_fillna_ffil.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date
20094,Alabama,339.65,1033,198.04,926,147.15,122,2013-12-27
20859,Alabama,339.65,1033,198.04,926,147.15,122,2013-12-28
21573,Alabama,339.75,1036,198.26,929,149.49,123,2013-12-29
22287,Alabama,339.75,1036,198.81,930,149.49,123,2013-12-30
22797,Alabama,339.42,1040,198.68,932,149.49,123,2013-12-31


In [20]:
statistics_df = pd.DataFrame(csv_df_sort_fillna_ffil.groupby(['State'],as_index=False).aggregate({'HighQ': {'HighQ_Mean': np.mean,'HighQ_Median':np.median,'HighQ_Mode':stats.mode,'HighQ_Std':np.std,'HighQ_Var':np.var},'LowQ':{'LowQ_Mean':np.mean,'LowQ_Median':np.median,'LowQ_Mode':stats.mode,'LowQ_Std':np.std,'LowQ_Var':np.var}}))
print type(statistics_df)
statistics_df = statistics_df.rename(columns={'':'State'})
print type(statistics_df)
statistics_df.columns = statistics_df.columns.droplevel(0)
statistics_df.head()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,State,HighQ_Mean,HighQ_Std,HighQ_Mode,HighQ_Var,HighQ_Median,LowQ_Std,LowQ_Mode,LowQ_Mean,LowQ_Var,LowQ_Median
0,Alabama,339.561849,1.539724,"([337.32], [10.0])",2.370749,340.10,2.073186,"([144.98], [216.0])",145.978508,4.298099,144.98
1,Alaska,291.482004,5.580105,"([285.65], [19.0])",31.137567,289.81,13.188305,"([403.33], [237.0])",394.653964,173.931381,403.33
2,Arizona,300.667483,2.015750,"([300.37], [10.0])",4.063247,300.46,3.961371,"([185.78], [213.0])",188.500134,15.692461,185.78
3,Arkansas,348.056147,6.866002,"([349.29], [11.0])",47.141986,348.21,1.172681,"([126.1], [219.0])",126.771269,1.375181,126.10
4,California,245.376125,1.727046,"([245.03], [8.0])",2.982686,245.31,1.598252,"([188.6], [211.0])",189.783586,2.554409,188.60


In [21]:
#let us now read demographics data into a dataframe 

In [22]:
demographics_df = pd.DataFrame.from_csv("Demographics_State.csv",header=0,index_col=False,sep=',')

In [23]:
demographics_df.head()

,region,total_population,percent_white,percent_black,percent_asian,percent_hispanic,per_capita_income,median_rent,median_age
0,alabama,4799277,67,26,1,4,23680,501,38.1
1,alaska,720316,63,3,5,6,32651,978,33.6
2,arizona,6479703,57,4,3,30,25358,747,36.3
3,arkansas,2933369,74,15,1,7,22170,480,37.5
4,california,37659181,40,6,13,38,29527,1119,35.4


In [24]:
demographics_df.shape

(51, 9)

In [25]:
demographics_df.columns

Index([u'region', u'total_population', u'percent_white', u'percent_black',
       u'percent_asian', u'percent_hispanic', u'per_capita_income',
       u'median_rent', u'median_age'],
      dtype='object')

In [26]:
#let us now read population data into a dataframe

In [27]:
population_df = pd.DataFrame.from_csv("Population_State.csv",header=0,index_col=False,sep=",")

In [28]:
population_df.head()

,region,value
0,alabama,4777326
1,alaska,711139
2,arizona,6410979
3,arkansas,2916372
4,california,37325068


In [29]:
population_df.shape

(51, 2)

Object `pd.describe` not found.


In [73]:
?population_df.describe()

In [30]:
population_df.columns

Index([u'region', u'value'], dtype='object')

In [31]:
#we are now merging demographic and population data into one single data frame
#to do this , we are using pandas merge method which allows us to join two data frames like how we do it in sql
# we are using inner join and on the column region

In [32]:
population_demographic_merge_df = pd.merge(demographics_df,population_df,how='inner',on='region')

In [33]:
population_demographic_merge_df.head()

,region,total_population,percent_white,percent_black,percent_asian,percent_hispanic,per_capita_income,median_rent,median_age,value
0,alabama,4799277,67,26,1,4,23680,501,38.1,4777326
1,alaska,720316,63,3,5,6,32651,978,33.6,711139
2,arizona,6479703,57,4,3,30,25358,747,36.3,6410979
3,arkansas,2933369,74,15,1,7,22170,480,37.5,2916372
4,california,37659181,40,6,13,38,29527,1119,35.4,37325068


In [34]:
population_demographic_merge_df.shape

(51, 10)

In [35]:
population_demographic_merge_df = population_demographic_merge_df.rename(columns={'region':'State'})

In [36]:
population_demographic_merge_df.head()

,State,total_population,percent_white,percent_black,percent_asian,percent_hispanic,per_capita_income,median_rent,median_age,value
0,alabama,4799277,67,26,1,4,23680,501,38.1,4777326
1,alaska,720316,63,3,5,6,32651,978,33.6,711139
2,arizona,6479703,57,4,3,30,25358,747,36.3,6410979
3,arkansas,2933369,74,15,1,7,22170,480,37.5,2916372
4,california,37659181,40,6,13,38,29527,1119,35.4,37325068


In [37]:
statistics_population_demographic_merge_df = pd.merge(population_demographic_merge_df,statistics_df,how='inner',on='State')

In [38]:
statistics_population_demographic_merge_df.head()

,State,total_population,percent_white,percent_black,percent_asian,percent_hispanic,per_capita_income,median_rent,median_age,value,HighQ_Mean,HighQ_Std,HighQ_Mode,HighQ_Var,HighQ_Median,LowQ_Std,LowQ_Mode,LowQ_Mean,LowQ_Var,LowQ_Median


In [39]:
#the reason why there are no entries here is because in the population demographic df the States are in lower case
# let us now change the States to lower case in statistics df too

In [40]:
statistics_df.head()

,State,HighQ_Mean,HighQ_Std,HighQ_Mode,HighQ_Var,HighQ_Median,LowQ_Std,LowQ_Mode,LowQ_Mean,LowQ_Var,LowQ_Median
0,Alabama,339.561849,1.539724,"([337.32], [10.0])",2.370749,340.10,2.073186,"([144.98], [216.0])",145.978508,4.298099,144.98
1,Alaska,291.482004,5.580105,"([285.65], [19.0])",31.137567,289.81,13.188305,"([403.33], [237.0])",394.653964,173.931381,403.33
2,Arizona,300.667483,2.015750,"([300.37], [10.0])",4.063247,300.46,3.961371,"([185.78], [213.0])",188.500134,15.692461,185.78
3,Arkansas,348.056147,6.866002,"([349.29], [11.0])",47.141986,348.21,1.172681,"([126.1], [219.0])",126.771269,1.375181,126.10
4,California,245.376125,1.727046,"([245.03], [8.0])",2.982686,245.31,1.598252,"([188.6], [211.0])",189.783586,2.554409,188.60


In [41]:
statistics_df['State'] = statistics_df['State'].str.lower()
statistics_df.head()

,State,HighQ_Mean,HighQ_Std,HighQ_Mode,HighQ_Var,HighQ_Median,LowQ_Std,LowQ_Mode,LowQ_Mean,LowQ_Var,LowQ_Median
0,alabama,339.561849,1.539724,"([337.32], [10.0])",2.370749,340.10,2.073186,"([144.98], [216.0])",145.978508,4.298099,144.98
1,alaska,291.482004,5.580105,"([285.65], [19.0])",31.137567,289.81,13.188305,"([403.33], [237.0])",394.653964,173.931381,403.33
2,arizona,300.667483,2.015750,"([300.37], [10.0])",4.063247,300.46,3.961371,"([185.78], [213.0])",188.500134,15.692461,185.78
3,arkansas,348.056147,6.866002,"([349.29], [11.0])",47.141986,348.21,1.172681,"([126.1], [219.0])",126.771269,1.375181,126.10
4,california,245.376125,1.727046,"([245.03], [8.0])",2.982686,245.31,1.598252,"([188.6], [211.0])",189.783586,2.554409,188.60


In [42]:
#now lets merge the data again, the reason we are merging all this data is we are able to look at one state on multiple levels.

In [43]:
stats_pop_dem_merge_df = pd.merge(population_demographic_merge_df,statistics_df,how='inner',on='State')

In [44]:
stats_pop_dem_merge_df.head()

,State,total_population,percent_white,percent_black,percent_asian,percent_hispanic,per_capita_income,median_rent,median_age,value,HighQ_Mean,HighQ_Std,HighQ_Mode,HighQ_Var,HighQ_Median,LowQ_Std,LowQ_Mode,LowQ_Mean,LowQ_Var,LowQ_Median
0,alabama,4799277,67,26,1,4,23680,501,38.1,4777326,339.561849,1.539724,"([337.32], [10.0])",2.370749,340.10,2.073186,"([144.98], [216.0])",145.978508,4.298099,144.98
1,alaska,720316,63,3,5,6,32651,978,33.6,711139,291.482004,5.580105,"([285.65], [19.0])",31.137567,289.81,13.188305,"([403.33], [237.0])",394.653964,173.931381,403.33
2,arizona,6479703,57,4,3,30,25358,747,36.3,6410979,300.667483,2.015750,"([300.37], [10.0])",4.063247,300.46,3.961371,"([185.78], [213.0])",188.500134,15.692461,185.78
3,arkansas,2933369,74,15,1,7,22170,480,37.5,2916372,348.056147,6.866002,"([349.29], [11.0])",47.141986,348.21,1.172681,"([126.1], [219.0])",126.771269,1.375181,126.10
4,california,37659181,40,6,13,38,29527,1119,35.4,37325068,245.376125,1.727046,"([245.03], [8.0])",2.982686,245.31,1.598252,"([188.6], [211.0])",189.783586,2.554409,188.60


In [45]:
stats_pop_dem_merge_df.shape

(51, 20)

In [46]:
#so now we have data for 51 states with 20 feature columns. 
#The data is ready

In [47]:
white_greater_black = stats_pop_dem_merge_df[stats_pop_dem_merge_df.percent_white > stats_pop_dem_merge_df.percent_black]
black_greate_white =  stats_pop_dem_merge_df[stats_pop_dem_merge_df.percent_white < stats_pop_dem_merge_df.percent_black]


In [48]:
white_greater_black.shape,black_greate_white.shape

((50, 20), (1, 20))

In [49]:
#given that the white population is high in almost every state, lets not do any comparison use the 
#percent of whites and blacks

In [50]:
#let us see if there is a correlation between per capita income and highQ_mean/lowQ_mean
correlation_btw_per_capita_highQ = stats.pearsonr(white_greater_black.per_capita_income,white_greater_black.HighQ_Mean)[0]
correlation_btw_per_capita_lowQ = stats.pearsonr(white_greater_black.per_capita_income,white_greater_black.LowQ_Mean)[0]

In [51]:
correlation_btw_per_capita_highQ,correlation_btw_per_capita_lowQ

(0.10372726217632158, 0.34288507101733851)

In [52]:
# from this we understand that the value of low quality weed is somewhat affected by per capita income compared to 
# high quality weed
# we can use the same way to compute correlations amognst other attributes too

In [53]:
#before we go any further , lets try to find some interesting growth in weed prices for each state 
csv_df_sort_fillna_ffil.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date
20094,Alabama,339.65,1033,198.04,926,147.15,122,2013-12-27
20859,Alabama,339.65,1033,198.04,926,147.15,122,2013-12-28
21573,Alabama,339.75,1036,198.26,929,149.49,123,2013-12-29
22287,Alabama,339.75,1036,198.81,930,149.49,123,2013-12-30
22797,Alabama,339.42,1040,198.68,932,149.49,123,2013-12-31


In [54]:
country_mean_highQ = csv_df_sort_fillna_ffil.groupby(['date'],as_index=False).aggregate({'HighQ':np.mean})

In [55]:
country_mean_highQ.head()

,date,HighQ
0,2013-12-27,335.498039
1,2013-12-28,335.494902
2,2013-12-29,335.582941
3,2013-12-30,335.490196
4,2013-12-31,335.505098


In [56]:
'''
plt.plot(country_mean_highQ.HighQ)
plt.xticks(range(len(country_mean_highQ.date)),country_mean_highQ.date)
plt.show()
'''

'\nplt.plot(country_mean_highQ.HighQ)\nplt.xticks(range(len(country_mean_highQ.date)),country_mean_highQ.date)\nplt.show()\n'

In [57]:
country_mean_highQ.min()

date     2013-12-27
HighQ      322.1294
dtype: object

In [58]:
#we just have dates, we need to generate calendar week for each date

In [59]:
from datetime import datetime
# date format in %Y-%m-%d
def get_calendar_week(date):
    calendar_week = datetime.strptime(date,"%Y-%m-%d").isocalendar()[1]
    return int(calendar_week)

In [60]:
# lets add calendar week to the data frame
csv_df_sort_fillna_ffil['Calendar_Week'] = csv_df_sort_fillna_ffil['date'].apply(get_calendar_week)

In [61]:
#lets check if it looks fine
csv_df_sort_fillna_ffil.head()

,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date,Calendar_Week
20094,Alabama,339.65,1033,198.04,926,147.15,122,2013-12-27,52
20859,Alabama,339.65,1033,198.04,926,147.15,122,2013-12-28,52
21573,Alabama,339.75,1036,198.26,929,149.49,123,2013-12-29,52
22287,Alabama,339.75,1036,198.81,930,149.49,123,2013-12-30,1
22797,Alabama,339.42,1040,198.68,932,149.49,123,2013-12-31,1


In [66]:
#now let us try to group at week level and see during which week it was the lowest and for a given state
country_week_highQ_df = pd.DataFrame(csv_df_sort_fillna_ffil.groupby(['Calendar_Week'],as_index=False).aggregate({'HighQ':{'HighQ_Mean':np.mean}}))
country_week_highQ_df.columns = country_week_highQ_df.columns.droplevel(0)
country_week_highQ_df = country_week_highQ_df.rename(columns={'':'Calendar_Week'})
country_week_highQ_df.head()
print country_week_highQ_df.min(),country_week_highQ_df.max()


Calendar_Week      1.000000
HighQ_Mean       326.614174
dtype: float64 Calendar_Week     52.000000
HighQ_Mean       333.778207
dtype: float64


In [67]:
#now let us check on which day it was the lowest across country
country_day_highQ_df = pd.DataFrame(csv_df_sort_fillna_ffil.groupby(['date'],as_index=False).aggregate({'HighQ':{'HighQ_Mean':np.mean}}))
country_day_highQ_df.columns = country_day_highQ_df.columns.droplevel(0)
country_day_highQ_df = country_day_highQ_df.rename(columns={'':'date'})
country_day_highQ_df.head()
print country_day_highQ_df.min(),country_day_highQ_df.max()

date          2013-12-27
HighQ_Mean      322.1294
dtype: object date          2015-06-11
HighQ_Mean      335.5829
dtype: object


In [68]:
# now let us check when people buy the most across the country 
country_week_highQN_df = pd.DataFrame(csv_df_sort_fillna_ffil.groupby(['Calendar_Week'],as_index=False).aggregate({'HighQN':{'HighQN_Mean':np.mean}}))
country_week_highQN_df.columns = country_week_highQN_df.columns.droplevel(0)
country_week_highQN_df = country_week_highQN_df.rename(columns={'':'Calendar_Week'})
country_week_highQN_df.head()
print country_week_highQN_df.min(),country_week_highQN_df.max()

Calendar_Week       1.000000
HighQN_Mean      1951.098039
dtype: float64 Calendar_Week      52.000000
HighQN_Mean      2522.322129
dtype: float64
